# Target Description Language

In [ ]:
from keckODL.target import Target, TargetList

In [ ]:
# Example: fully specify target.  Coordinates in decimal degrees.
t1 = Target(name='M42', RA=82.5, Dec=-5, frame='icrs',
            PMRA=10, PMDec=-10, epoch=1975.5,
            rotmode='PA', PA=90,
            objecttype='science',
            RAOffset=8.4, DecOffset=-4.5,
            acquisition='guider: offset',
            mag={'V': 4.3, 'K': 6.7},
            comment='Star formation is cool',
            wrap='shortest',
           )
t1

In [ ]:
# Example: fully specify target.  Coordinates in hmsdms.
t2 = Target(name='M31', RA='00:42:44.33', Dec='41:16:07.50',
            rotmode='PA', PA=32,
            RAOffset=12.1, DecOffset=-1.9,
            objecttype='science',
            acquisition='guider: offset',
            mag={'V': 2.3, 'K': 8.7},
            comment='Galaxies are boring',
            wrap='north',
           )
t2

In [ ]:
# Example: Resolve target by name.
t3 = Target('NGC1333',
            rotmode='PA', PA=22.5,
            objecttype='science',
            acquisition='blind',
            comment='Low mass star formation is the coolest')
t3

In [ ]:
# Example: Resolve target by name, set vertical angle mode, set obstype
t4 = Target('Feige110',
            objecttype='flux standard',
            rotmode='vertical',
            acquisition='guider: bright',
            comment='I guess we need a standard')
t4

In [ ]:
# Build a target list, write as TDL TAML file and as starlist
tl = TargetList([t1, t2, t3, t4])
tl.write('targets.txt')
tl.write_starlist('starlist.txt')
tl

In [ ]:
# Read in a target list from the TDL YAML file
newtl = TargetList().read('targets.txt')
newtl

In [ ]:
from astropy.time import Time
tl.set_obstime(Time(2045.11, format='decimalyear'))
tl

# Observing Sequences

## KCWI Example

In [1]:
from keckODL.kcwi import KCWIblueDetectorConfig
from keckODL.kcwi import KCWIblueConfig
from keckODL.offset import ABBA, Stare, StarSkyStar
from keckODL.sequence import SequenceElement, Sequence

In [2]:
kcwi_1800s = KCWIblueDetectorConfig(exptime=1800, binning='2x2')
kcwi_1800s.write('kcwi_1800s.txt')
print(kcwi_1800s)

1800.0s (10)


In [3]:
med_slicer_4800 = KCWIblueConfig(slicer='medium', grating='BH3', cwave=4800)
print(med_slicer_4800)

medium BH3 4800 A


In [4]:
skyoffset = StarSkyStar(dx=30, dy=30)
print(skyoffset)

StarSkyStar (30 30)


In [5]:
skyoffset

Frame: SkyFrame
 dx(")| dy(")| dr(deg)|    name
------|------|--------|--------
  +0.0|  +0.0|    +0.0|    star
 +30.0| +30.0|    +0.0|     sky
  +0.0|  +0.0|    +0.0|    star

In [6]:
myseq = Sequence([SequenceElement(pattern=skyoffset,
                                  detconfig=kcwi_1800s,
                                  instconfig=med_slicer_4800,
                                  repeat=3),
                  ])
print(myseq)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
StarSkyStar (30 30)|     3 |1800.0s (10)                 |medium BH3 4800 A                            


In [7]:
myseq.estimate_time()

Shutter Open Time: 16200 s (4.5 hrs)
Wall Clock Time: 16200 s (4.5 hrs)


## MOSFIRE Example

In [8]:
from keckODL.mosfire import MOSFIREDetectorConfig
from keckODL.mosfire import MOSFIREConfig
from keckODL.offset import ABBA, Long2pos, Stare
from keckODL.sequence import SequenceElement, Sequence

In [9]:
# detector configs
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_180s.write('mosfire_180s.txt')
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
mosfire_120s.write('mosfire_120s.txt')
print(mosfire_180s)
print(mosfire_120s)

180.0s (MCDS16, 1 coadds)
120.0s (MCDS16, 1 coadds)


In [10]:
# instrument configs
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
mosfire_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='my_MOS_mask')
mosfire_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='my_MOS_mask')
mosfire_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='my_MOS_mask')
print(mosfire_Y)
print(mosfire_J)
print(mosfire_H)
print(mosfire_K)

my_MOS_mask Y-spectroscopy
my_MOS_mask J-spectroscopy
my_MOS_mask H-spectroscopy
my_MOS_mask K-spectroscopy


In [11]:
abba = ABBA(offset=1.5)
print(abba)

ABBA (1.5)


In [12]:
abba

Frame: SkyFrame
 dx(")| dy(")| dr(deg)|    name
------|------|--------|--------
  +0.0|  +1.5|    +0.0|       A
  +0.0|  -1.5|    +0.0|       B
  +0.0|  -1.5|    +0.0|       B
  +0.0|  +1.5|    +0.0|       A

In [13]:
# sequence
myseq_mosfire = Sequence([SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_Y, repeat=5),
                          SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_J, repeat=8),
                          SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_H, repeat=8),
                          SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_K, repeat=5),
                         ])
print(myseq_mosfire)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
ABBA (1.5)         |     5 |180.0s (MCDS16, 1 coadds)    |my_MOS_mask Y-spectroscopy                   
ABBA (1.5)         |     8 |120.0s (MCDS16, 1 coadds)    |my_MOS_mask J-spectroscopy                   
ABBA (1.5)         |     8 |120.0s (MCDS16, 1 coadds)    |my_MOS_mask H-spectroscopy                   
ABBA (1.5)         |     5 |180.0s (MCDS16, 1 coadds)    |my_MOS_mask K-spectroscopy                   


In [14]:
myseq_mosfire.estimate_time()

Shutter Open Time: 14880 s (4.1 hrs)
Wall Clock Time: 14880 s (4.1 hrs)


In [15]:
cals_I_need = mosfire_Y.cals()
cals_I_need.extend( mosfire_J.cals() )
cals_I_need.extend( mosfire_H.cals() )
cals_I_need.extend( mosfire_K.cals() )
print(cals_I_need)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
Stare              |     7 |11.0s (CDS, 1 coadds)        |my_MOS_mask Y-spectroscopy domeflatlamp=True 
Stare              |     7 |11.0s (CDS, 1 coadds)        |my_MOS_mask J-spectroscopy domeflatlamp=True 
Stare              |     7 |11.0s (CDS, 1 coadds)        |my_MOS_mask H-spectroscopy domeflatlamp=True 
Stare              |     7 |11.0s (CDS, 1 coadds)        |my_MOS_mask K-spectroscopy domeflatlamp=True 
Stare              |     7 |11.0s (CDS, 1 coadds)        |my_MOS_mask K-spectroscopy domeflatlamp=False
Stare              |     2 |1.0s (CDS, 1 coadds)         |my_MOS_mask K-spectroscopy arclamp=Ne        
Stare              |     2 |1.0s (CDS, 1 coadds)         |my_MOS_mask K-spectroscopy arclamp=Ar        


In [16]:
cals_I_need.estimate_time()

Shutter Open Time: 389 s (0.1 hrs)
Wall Clock Time: 389 s (0.1 hrs)


## MOSFIRE Example 2

In [17]:
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_3x0.7')
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_3x0.7')
singleobj_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='long2pos')
singleobj_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='long2pos')
print(singleobj_Y)
print(singleobj_J)
print(singleobj_H)
print(singleobj_K)

longslit_3x0.7 Y-spectroscopy
longslit_3x0.7 J-spectroscopy
long2pos H-spectroscopy
long2pos K-spectroscopy


In [18]:
long2pos = Long2pos()
print(long2pos)

long2pos


In [19]:
long2pos

Frame: MOSFIRE Detector
 dx(")| dy(")| dr(deg)|    name
------|------|--------|--------
 +45.0| -23.0|    +0.0|       A
 +45.0|  -9.0|    +0.0|       B
 -45.0|  +9.0|    +0.0|       A
 -45.0| +23.0|    +0.0|       B

In [20]:
# sequence
myseq2_mosfire = Sequence([SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=singleobj_Y, repeat=2),
                           SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=singleobj_J, repeat=2),
                           SequenceElement(pattern=long2pos, detconfig=mosfire_120s, instconfig=singleobj_H, repeat=2),
                           SequenceElement(pattern=long2pos, detconfig=mosfire_180s, instconfig=singleobj_K, repeat=2),
                          ])
print(myseq2_mosfire)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
ABBA (1.5)         |     2 |180.0s (MCDS16, 1 coadds)    |longslit_3x0.7 Y-spectroscopy                
ABBA (1.5)         |     2 |120.0s (MCDS16, 1 coadds)    |longslit_3x0.7 J-spectroscopy                
long2pos           |     2 |120.0s (MCDS16, 1 coadds)    |long2pos H-spectroscopy                      
long2pos           |     2 |180.0s (MCDS16, 1 coadds)    |long2pos K-spectroscopy                      


In [21]:
myseq2_mosfire.estimate_time()

Shutter Open Time: 4800 s (1.3 hrs)
Wall Clock Time: 4800 s (1.3 hrs)


In [22]:
cals_I_need = singleobj_Y.cals()
cals_I_need.extend(singleobj_J.cals())
cals_I_need.extend(singleobj_H.cals())
cals_I_need.extend(singleobj_K.cals())
print(cals_I_need)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
Stare              |     7 |11.0s (CDS, 1 coadds)        |longslit_3x0.7 Y-spectroscopy domeflatlamp=True
Stare              |     7 |11.0s (CDS, 1 coadds)        |longslit_3x0.7 J-spectroscopy domeflatlamp=True
Stare              |     7 |11.0s (CDS, 1 coadds)        |long2pos H-spectroscopy domeflatlamp=True    
Stare              |     7 |11.0s (CDS, 1 coadds)        |long2pos K-spectroscopy domeflatlamp=True    
Stare              |     7 |11.0s (CDS, 1 coadds)        |long2pos K-spectroscopy domeflatlamp=False   
Stare              |     2 |1.0s (CDS, 1 coadds)         |long2pos K-spectroscopy arclamp=Ne           
Stare              |     2 |1.0s (CDS, 1 coadds)         |long2pos K-spectroscopy arclamp=Ar           


In [23]:
cals_I_need.estimate_time()

Shutter Open Time: 389 s (0.1 hrs)
Wall Clock Time: 389 s (0.1 hrs)
